In [1]:
import math
import re
import os

In [2]:
headerFile = './std/header.bin'
#binFile = 'rcv.bin'
binFile = 'nRU_90x256.bin'

ufpFile = 'rcv.ufp'

In [3]:
hfdata = None
with open(headerFile,'rb') as f:
    hfdata = f.readlines()

In [4]:
hfdata

[b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00']

In [7]:
hfdata[0][:17]

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa'

In [17]:
bytes([hfdata[0][20]]),hfdata[0][21]

(b'1', 225)

In [5]:
len(hfdata[0])

64

In [22]:
nhf_data = [bytes([11]*63 + [12])]

In [23]:
nhf_data

[b'\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0c']

In [24]:
test_bin = 'test.bin'

In [25]:
with open(test_bin,'wb') as f:
    f.writelines(nhf_data)

In [26]:
test_data = None
with open(test_bin,'rb') as f:
    test_data = f.readlines()

In [27]:
test_data

[b'\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0c']

In [30]:
rdata = []
for line in open(test_bin,'rb'):
    rdata.append(line)

In [31]:
rdata

[b'\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0c']

In [34]:
len(rdata),len(rdata[0])

(1, 64)

In [35]:
rdata = []
for line in open(headerFile,'rb'):
    rdata.append(line)

In [36]:
rdata

[b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00']

In [37]:
len(rdata),len(rdata[0])

(1, 64)

In [62]:
def archiveProcess(fpath, page=256, sbase=256):
    rdata = []
    #sectorBytes = page * sbase
    ftype = os.path.splitext(fpath)[1] if fpath else None
    print(ftype)
    if ftype and ftype == '.bin':
        for line in open(fpath, 'rb'):
            rdata.append(line.strip())
        #with open(fpath,'rb') as f:
        #    rdata = f.readlines()
    rcluster = b''.join(rdata)
    lrc = len(rcluster)
    #print(rdata)
    print('RAWDATA: {} - Size: {}'.format(fpath, lrc))
    #print('FileContent: {}'.format(rcluster[:256]))
    #pacN = math.ceil(lrc/page)
    #ff_pad = bytes.fromhex('ff' * (pacN * 256 - lrc))

    sectors = math.ceil(lrc/(page*sbase))   
    sectorBytes = sectors * page * sbase
    
    ff_pad = bytes.fromhex('ff' * (sectorBytes - lrc))
    
    sendContent = b''.join([rcluster, ff_pad])
    lsc = len(sendContent)
    
    #sectors = math.ceil(lsc/sectorBytes) - 1
    sendPacs = [sendContent[i*256:(i+1)*256] for i in range(page*sectors)]
 
    print('SendInfo - Size: {} $$ PacketN: {} $$ Sectors: {}'.format(lsc, len(sendPacs), sectors))
    
    return sendPacs, sectors

In [59]:
test_bin

'test.bin'

In [60]:
archiveProcess(test_bin)

.bin
[b'']
RAWDATA: test.bin - Size: 0
SendInfo - Size: 0 $$ PacketN: 0 $$ Sectors: 0


([], 0)

In [63]:
archiveProcess(binFile)

.bin
RAWDATA: rcv.bin - Size: 1483603
SendInfo - Size: 1507328 $$ PacketN: 5888 $$ Sectors: 23


([b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00 \x00 \x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf0\xa1\x00\x07 \x001\xa1\x06(1A=\x001a\t\xee1\xc2\x04\x00\x80\x930\xe1\x00\xcf0\xc1\x00\x81 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\x81<\xc81\x81\x08\x814!\x00\x002\x01\x00\x1f1\xe1\xff\xff3!\x00\x053A\x00\x04 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\xa1\x1b\xe23\xc2\x00\x00\x00\x00 \x00 \x000"\x00\x00\x00\x000\xa1\x00\x01P`\x00\x0bR\xa9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x10\x00\x00\x00\x0

In [64]:
archiveProcess(headerFile)

.bin
RAWDATA: ./std/header.bin - Size: 64
SendInfo - Size: 65536 $$ PacketN: 256 $$ Sectors: 1


([b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff',
  b'\xff\xff\xff\xff\xf

In [38]:
bindata[0][:256]

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00 \x00 \x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf0\xa1\x00\x07 \x001\xa1\x06(1A=\x001a\t\xee1\xc2\x04\x00\x80\x930\xe1\x00\xcf0\xc1\x00\x81 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\x81<\xc81\x81\x08\x814!\x00\x002\x01\x00\x1f1\xe1\xff\xff3!\x00\x053A\x00\x04 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\xa1\x1b\xe23\xc2\x00\x00\x00\x00 \x00 \x000"\x00\x00\x00\x000\xa1\x00\x01P`\x00\x0bR\xa9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [68]:
len(bindata)

2224

In [66]:
bindata = None
with open(binFile,'rb') as f:
    bindata = f.readlines()

In [67]:
len(bindata[0].strip()), len(bindata[0])

(3374, 3375)

In [70]:
bindata[0].strip()[:256]

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf0\xa1\x00\x07 \x001\xa1\x06(1A=\x001a\t\xee1\xc2\x04\x00\x80\x930\xe1\x00\xcf0\xc1\x00\x81 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\x81<\x121\x81\x08\x814!\x00\x002\x01\x00\x1f1\xe1\xff\xff3!\x00\x053A\x00\x043\x01\x01\x002a\x00\x002\x81\x00\x002\xa1\x00\x002\xc1\x00\x002\xe1\x00\x003\xa1\x1b\xe23\xc2\x00\x00\x00\x00 \x00 \x000"\x00\x00\x00\x000\xa1\x00\x01P`\x00\x0bR\xa9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x10\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [47]:
bindata[0]

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00 \x00 \x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf0\xa1\x00\x07 \x001\xa1\x06(1A=\x001a\t\xee1\xc2\x04\x00\x80\x930\xe1\x00\xcf0\xc1\x00\x81 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\x81<\xc81\x81\x08\x814!\x00\x002\x01\x00\x1f1\xe1\xff\xff3!\x00\x053A\x00\x04 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x00 \x003\xa1\x1b\xe23\xc2\x00\x00\x00\x00 \x00 \x000"\x00\x00\x00\x000\xa1\x00\x01P`\x00\x0bR\xa9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00

In [44]:
bindata[0][:32], bindata[0][65:75] , len(b''.join([bindata[0][:10], bindata[0][65:75]]))

(b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@',
 b'\x00 \x00\xff\xff\xff\xff\xff\xff\xff',
 20)

In [45]:
ufpdata = None
with open(ufpFile,'rb') as f:
    ufpdata = f.readlines()

In [9]:
ufpdata[:10], ufpdata[0], len(ufpdata[0]), len(ufpdata), len(ufpdata[0].strip())

([b'ffffffffffffffffffffffffffffffff\r\n',
  b'5599aa668c87ffff4c8600004c810002\r\n',
  b'4c8500224c8300004c8700000c850000\r\n',
  b'cc8084004c8000fa0c85007004000400\r\n',
  b'04000400ffffffffffffffffffffffff\r\n',
  b'ffffffff5599aa660c8500e004008c85\r\n',
  b'60148c82bc008c8690778c43200001c9\r\n',
  b'0c8700f30c8300810400040004000400\r\n',
  b'04000400040004000400040004000400\r\n',
  b'04000400040004000400cc813c138c81\r\n'],
 b'ffffffffffffffffffffffffffffffff\r\n',
 34,
 92866,
 32)

In [10]:
sum(len(d.strip()) for d in bindata)

1483603

In [11]:
sum(len(d.strip()) for d in ufpdata)

2971712

In [12]:
ps0 = ufpdata[0]
ps1 = ufpdata[1]

In [13]:
ps0,ps1

(b'ffffffffffffffffffffffffffffffff\r\n',
 b'5599aa668c87ffff4c8600004c810002\r\n')

In [14]:
b''.join([ps0,ps1]), len(b''.join([ps0,ps1]))

(b'ffffffffffffffffffffffffffffffff\r\n5599aa668c87ffff4c8600004c810002\r\n',
 68)

In [15]:
math.ceil(101/4)

26

In [16]:
bytes.fromhex('ff'*4)

b'\xff\xff\xff\xff'

In [49]:
first16_fpath = 'e:/proj_Enoch/Hasco/xbEnetData/assemble/16st_11.fd'

In [37]:
os.path.split(first16_fpath)

('e:/proj_Enoch/Hasco/xbEnetData/assemble', '16st.fd')

In [38]:
os.path.split(ufpFile)

('', 'rcv.ufp')

In [50]:
os.path.splitext(first16_fpath)[1] == '.fd'

True

In [56]:
os.path.splitext(ufpFile)

('rcv', '.ufp')

In [46]:
fpath = None
a = os.path.splitext(fpath) if fpath else None

In [53]:
a = 3
1< a < 4

True

In [57]:
a = [i for i in range(16)]

In [60]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [61]:
a[:-4]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [63]:
c = bytes([0,1,2,3,4,5,6,7])

In [64]:
c

b'\x00\x01\x02\x03\x04\x05\x06\x07'

In [69]:
b''.join([c, bytes([0]*3)])

b'\x00\x01\x02\x03\x04\x05\x06\x07\x00\x00\x00'

In [71]:
bytes([512%256])

b'\x00'

In [83]:
a = 0 if a == 1 else a

In [84]:
a

0

In [72]:
a = 1
b = 0

In [75]:
a = a and b

In [77]:
a = a or b

In [80]:
a = not a or b

In [81]:
a

0

In [55]:
os.listdir('./')

['.ipynb_checkpoints',
 'rcv.bgn',
 'rcv.bin',
 'rcv.bit',
 'rcv.cfi',
 'rcv.drc',
 'rcv.mcs',
 'rcv.ncd',
 'rcv.prm',
 'rcv.ufp',
 'rcv_bitgen.xwbt',
 'std',
 'Untitled.ipynb',
 'usage_statistics_webtalk.html',
 'webtalk.log',
 'xilinx_device_details.xml',
 '_xmsgs']

In [17]:
def archiveProcess(fpath, page=256, sbase=256):
    rdata = []
    for line in open(fpath, 'rb'):
        rdata.append(line.strip())
    rcluster = b''.join(rdata)
    lrc = len(rcluster)
    print('File: {} - Size: {}'.format(fpath, lrc))
    
    pacN = math.ceil(lrc/page)
    ff_pad = bytes.fromhex('ff' * (pacN * 256 - lrc))
    
    sendContent = b''.join([rcluster, ff_pad])
    lsc = len(sendContent)
    
    sendPacs = [sendContent[i*256:(i+1)*256] for i in range(pacN)]
    
    sectorBytes = page * sbase
    
    sectors = math.ceil(lsc/sectorBytes)
 
    print('SendInfo - Size: {} $$ PacketN: {} $$ Sectors: {}'.format(lsc, pacN, sectors))
    
    return sendPacs, sectors

In [18]:
archiveProcess(headerFile)

File: ./std/header.bin - Size: 64
SendInfo - Size: 256 $$ PacketN: 1 $$ Sectors: 1


([b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xaa\x99Uf1\xe1\xff\xff2a\x00\x002\x81\x03@2\xa1\x00\x002\xc1\x03\x012\xe1\x00\x000\xa1\x00\x003\x01!\x002\x01\x00_0\xa1\x00\x0e \x00 \x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'],
 1)

In [19]:
cmd_type = [
            'NORMAL_UPDOWN',
            'HEADER_UPDATE',
            'MTBOOT_UPDATE',
            'GOLDEN_UPDATE',
            'CHANGE_CONFIG',
            'RDBACK_HEADER',
            'PICKUP_CONFIG'
            ]
cmd_en = 1
bsd_en = 0
newpkt = 0
cmd_in = cmd_type.index('CHANGE_CONFIG')
sens_en = 1
sectN = 0
adda = '40'

word0_b = [cmd_en*128, 0, bsd_en*32, newpkt*16, cmd_in]
word0_h = sum(word0_b)
print(word0_h)

cword0 = bytes([word0_h])
sens_cmd = bytes([sens_en])
sectors = bytes([sectN])
sadda = bytes.fromhex(adda)
reserved = bytes([0])

cmd_tb = {
        0: cword0,
        1: sens_cmd,
        2: sectors,
        3: sadda,
        4: reserved,
        5: reserved, 
        6: reserved,
        7: reserved
    }

132


In [20]:
cmd_tb

{0: b'\x84',
 1: b'\x01',
 2: b'\x00',
 3: b'@',
 4: b'\x00',
 5: b'\x00',
 6: b'\x00',
 7: b'\x00'}

In [21]:
p10 = bytes.fromhex(h16s[10])

NameError: name 'h16s' is not defined

In [126]:
p10[:16]

b'UUU\xd5\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [164]:
cmd = b''.join([v for k,v in cmd_tb.items()])

In [165]:
cmd

b'\x84\x01\x00@\x00\x00\x00\x00'

In [166]:
cmd_type[cmd[0]%16]

'CHANGE_CONFIG'

In [147]:
b''.join([p10[:8],cmd])

b'UUU\xd5\n\x00\x00\x00\x80\x01\x00@\x00\x00\x00\x00'

In [22]:
def cmdPacketProc(cmd_tb,sdata,pac10,preamble=4):    
    new_p10 = None
    cmd = b''.join([v for k,v in cmd_tb.items()]) 
    if preamble!=0:
        cmd_stp = preamble + 4
        cmd_end = cmd_stp + 8
        dat_stp = cmd_end
        dat_end = dat_stp + 256
        new_p10 = [pac10[:cmd_stp], cmd, sdata, pac10[dat_end:]]
    return b''.join(new_p10)

In [23]:
sda = bytes([0]*256)

In [24]:
p10[:64]

NameError: name 'p10' is not defined

In [157]:
cmdPacketProc(cmd_tb,sda, p10)[:64]

b'UUU\xd5\n\x00\x00\x00\x80\x01\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [102]:
h16s = gen_first16_PKT()

16 2905


In [106]:
len(h16s[10])

2905

In [113]:
1040//256, 1040%256

(4, 16)

In [170]:
input('Please Type CMD Wanted:')

Please Type CMD Wanted:cmd


'cmd'

In [25]:
def gen_first16_PKT(pkt_len=1454,maxPanelNum=256,ctrlWords=None,panelControl=None,startPoint=None,cmd=None,OnlyPayload=False):

    pattern1 = re.compile('0x')
    pattern2 = re.compile(',')
    pattern3 = re.compile('0b')
    
    #hexPKT1_fpath = 'e:/proj_Enoch/Hasco/xbEnetData/hasco/1.txt'
    #hexPKT2_fpath = 'e:/proj_Enoch/Hasco/xbEnetData/hasco/2.txt'
    first16_fpath = 'e:/proj_Enoch/Hasco/xbEnetData/assemble/16st.fd'
    h16b_path = 'e:/proj_Enoch/Hasco/xbEnetData/assemble/16st.bin'
    #first16_fpath = '/mnt/hgfs/Hasco/xbEnetData/assemble/16st.fd'
    #dframe_fpath  = '/mnt/hgfs/Hasco/xbEnetData/dframe/'
    dframe_fpath  = 'e:/proj_Enoch/Hasco/xbEnetData/dframe/'
    #exists_fnames = os.listdir(eth_dir)
    #exists_indexs = [int(fn.split('.')[0]) for fn in exists_fnames]
    #exists_fpaths = [eth_dir + fn for fn in exists_fnames]
    pkt16st = []
    if os.path.exists(first16_fpath):
        for line in open(first16_fpath):
            pkt16st += [line]
        if len(pkt16st) != 16:
            pkt16st = []
    print(len(pkt16st),len(pkt16st[0]))
        
    for pn in range(16):
        csum = 0
        data = ['55','55','55','d5']
        pn_hstr = pattern1.sub('',hex(pn))
        pn_hex = '0'*(4-len(pn_hstr)) + pn_hstr
        data += [pn_hex[2:], pn_hex[:2] , '00','00']
        #print('pn: ',data)
        
        payload = ''
        if pn ==1 or pn == 2:
            
            if os.path.exists(os.path.join(dframe_fpath,str(pn)+'.txt')) and len(pkt16st) != 16:
                with open(os.path.join(dframe_fpath,str(pn)+'.txt'),'r',encoding='UTF-8') as f:
                    payload = f.readline().strip()[8*2:(pkt_len-4)*2]
            elif len(pkt16st) == 16:
                payload = pkt16st[pn].strip()[8*2:(pkt_len-4)*2]
            else:
                payload = '00' * (pkt_len - 12)                   
        
        if pn == 1:
            
            ctrl_data = [payload[i*2:(i+1)*2] for i in range(32)]
            ctrl_str = ''
            if isinstance(ctrlWords,dict): 
                for wn in range(32):
                    if wn in ctrlWords:
                        ctrl_data[wn] = ctrlWords[wn]
                #data += ctrl_data
            ctrl_str = ''.join(ctrl_data)
            data += ctrl_data
            ctrl_sum_str = pattern1.sub('',hex(sum(bytes.fromhex(ctrl_str))))
            ctrl_sum_hex = ('0' * (2 - len(ctrl_sum_str)) + ctrl_sum_str)[-2:] 
            data += [ctrl_sum_hex]
            data += ['00'] * (pkt_len - 4 - 4 - 33 - maxPanelNum * 2 - 4)
            pc_stidx = len(payload) - maxPanelNum * 4
            pc_data = [payload[pc_stidx+i*2:pc_stidx+(i+1)*2] for i in range(maxPanelNum)]
            if isinstance(panelControl,dict):
                for panelNum in range(maxPanelNum):
                    if panelNum in panelControl:
                        pc_data[panelNum] = panelControl[panelNum]
                #pc_data += [panelControl[panelNum] if wn in panelControl else '00' for panelNum in range(maxPanelNum)]
            data += pc_data
            data += ['00'] * maxPanelNum
            #print('Packet {} - Len(No Csum): {}'.format(pn,len(data)))
            
        elif pn == 2:
                       
            stp_data = [payload[i*2:(i+1)*2] for i in range(maxPanelNum*4)]
            if isinstance(startPoint,dict): 
                for panelNum in startPoint:
                    h_stp, v_stp = startPoint[panelNum]
                    h_stpr, v_stpr = pattern1.sub('',hex(h_stp)), pattern1.sub('',hex(v_stp))
                    h_stp16, v_stp16 = ('0'*(4-len(h_stpr)) + h_stpr)[-4:], ('0'*(4-len(v_stpr)) + v_stpr)[-4:]
                    h_stpL, h_stpH, v_stpL, v_stpH = h_stp16[2:], h_stp16[:2], v_stp16[2:],  v_stp16[:2]
                    startPoint[panelNum] = [h_stpL, h_stpH, v_stpL, v_stpH]
                    stp_data[panelNum*4]   = startPoint[panelNum][0]
                    stp_data[panelNum*4+1] = startPoint[panelNum][1]
                    stp_data[panelNum*4+2] = startPoint[panelNum][2]
                    stp_data[panelNum*4+3] = startPoint[panelNum][3]
            data += stp_data
            data += ['00'] * (pkt_len - 12 - maxPanelNum * 4)
        else:
            data += ['00'] * (pkt_len - 12)
            
        csum_str = pattern1.sub('',hex(sum(bytes.fromhex(''.join(data[4:])))))
        csum_hex = ('0'*(4-len(csum_str)) + csum_str)[-4:]
        
        data += [csum_hex[2:], csum_hex[:2], '00','00']
        
        if(len(pkt16st)) != 16:
            pkt16st += [''.join(data) + '\n']
        elif pn == 1 or pn == 2:
            pkt16st[pn] = ''.join(data) + '\n'
            
        #print('{} frame is done, dframe len : {}'.format(pn,len(''.join(data))//2))
    #print(len(pkt16st),len(pkt16st[0]))    
    with open(first16_fpath,'w',encoding='UTF-8') as fh:
        #fh.writelines('')
        fh.writelines(pkt16st)
    return pkt16st 

In [26]:
from scapy.all import *

In [27]:
def pack_callback(packet):
    #print ( packet.show() )
    #if packet['Ether'].payload:
    #    print (packet['Ether'].src)
    #    print (packet['Ether'].dst)
    #    print (packet['Ether'].type)
    #if packet['IP'] and packet['IP'].src == dst_ip:
    #    print ( packet.show() )
    #if packet['UDP'].payload:
    #    print (packet['Ether'].src)
    #    print (packet['Ether'].dst)
    #    print (packet['Ether'].type)
    #if packet['ARP'].payload:
    #    print (packet['ARP'].psrc)
    #    print (packet['ARP'].pdst)
    #    print (packet['ARP'].hwsrc)
    #    print (packet['ARP'].hwdst)
    pass
    time.sleep(2)

In [28]:
dst_mac = 'AA:BB:CC:DD:EE:FF'
src_mac = '77:88:77:88:77:88'
dst_ip = '176.16.44.40'
src_ip = '192.168.226.1'
#ip_len = pkt_len + 28
dst_port = 34501
src_port = 6400

In [29]:
def ackPacketScratch(packet):
    payload = packet['Raw'].load
    print(payload,type(payload))

In [30]:
filterstr="src host 176.16.44.40"
iface = 'Killer E2500 Gigabit Ethernet Controller'
#sniff(filter=filterstr, prn=pack_callback, lfilter=ackPacketScratch,iface=iface, count=1)
sniff(store =0, filter=filterstr, lfilter=ackPacketScratch,iface=iface, count=1)

b'UU\xd5\x03\x00\x00\x00O\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

b'UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU

b'UU\xd5\x03\x00\x00\x00O\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [31]:
input??

In [34]:
#print('Choose CMD U Wanted: ')
n = 0
while n < 10:
    i = input('Choose CMD U Wanted: ')
    print('Got {}'.format(i))
    n += 1

Choose CMD U Wanted: 0
Got 0
Choose CMD U Wanted: 1
Got 1
Choose CMD U Wanted: 2
Got 2
Choose CMD U Wanted: 3
Got 3
Choose CMD U Wanted: 4
Got 4
Choose CMD U Wanted: 5
Got 5
Choose CMD U Wanted: 6
Got 6
Choose CMD U Wanted: 7
Got 7
Choose CMD U Wanted: 8
Got 8
Choose CMD U Wanted: 9
Got 9


In [86]:
cmd_l = [1,0,1,1,1]
cmd_b = [cmd_l[0]*128, 0, cmd_l[2]*32, cmd_l[3]*16, cmd_l[4]]
cmd = sum(cmd_b)

In [87]:
cmd

177

In [88]:
payload = bytes([cmd])

In [94]:
payload[0]

177

In [96]:
ack_in = payload[0]
#sectN_rtn  = payload[10]
cmd_din_rtn = ack_in%16
cst_rtn = ack_in//16
cmd_en_rtn = cst_rtn//8
phase_rtn = cst_rtn%8//4
bsd_en_rtn = cst_rtn%4//2
newpkt_rtn = cst_rtn%2

In [100]:
cmd_en_rtn, phase_rtn, bsd_en_rtn, newpkt_rtn, cmd_din_rtn

(1, 0, 1, 1, 1)

In [1]:
print('{:#x}'.format(15))

0xf
